In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
import numpy as np
from datasets import load_metric, load_dataset, Dataset

In [5]:
import pandas as pd
import os

In [6]:
import torch
#torch.cuda.is_available()

In [7]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [9]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved to /home/hilhag/.huggingface/token
Login successful


In [12]:
df = pd.read_csv("test.csv", delimiter=",", quotechar="|")

In [13]:
tweets = Dataset.from_pandas(df)
tweets = tweets.train_test_split(test_size=0.1)

In [14]:
#scandisent = Dataset.from_pandas(df)
#scandisent = scandisent.train_test_split(test_size=0.1)

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("KB/bert-base-swedish-cased", num_labels=3)

Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at KB/bert-base-swedi

In [16]:
print(model.classifier.parameters)

<bound method Module.parameters of Linear(in_features=768, out_features=3, bias=True)>


In [17]:
#model

In [18]:
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [19]:
#len(list(set(train) & set(test)))

In [20]:
small_train_dataset = tweets["train"].shuffle(seed=42).select([i for i in list(range(4000))])
small_test_dataset = tweets["test"].shuffle(seed=42).select([i for i in list(range(500))])

In [21]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", model_max_length=512)

In [22]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [23]:
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
repo_name = "finetuning-sentiment-model"

In [26]:
tokenized_test

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 500
})

In [27]:
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=4,
   per_device_eval_batch_size=4,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=False,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [28]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/hilhag/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4000
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2000
  Number of trainable parameters = 124693251
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using 

Step,Training Loss


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
trainer.evaluate()